In [1]:
import torch
import torchvision 
import torch.nn as nn 
import torch.optim as optim
import torch.functional as F
import torchvision.transforms as transforms
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train_dataset = torchvision.datasets.CIFAR10(root='./data/' , train = True , transform=transforms.ToTensor() , download= True)
test_dataset = torchvision.datasets.CIFAR10(root='./data/' , train = False , transform=transforms.ToTensor() , download= True)

image , label = train_dataset[0]
print(image.size())#3色の32 * 32のデータになっている。
print(label)

Extracting ./data/cifar-10-python.tar.gz to ./data/
Files already downloaded and verified
torch.Size([3, 32, 32])
6


In [3]:
train_dataset

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data/
    Split: Train
    StandardTransform
Transform: ToTensor()

In [4]:
#DataLoaderはバッチサイズ分の画像と正解ラベルをdatasetから取り出す
train_loader = torch.utils.data.DataLoader(dataset= train_dataset,
                                           batch_size = 64 ,
                                           shuffle = True ,
                                           num_workers = 2)

test_loader = torch.utils.data.DataLoader(dataset= test_dataset,
                                           batch_size = 64 ,
                                           shuffle = False ,
                                           num_workers = 2)

In [5]:
for images , labels in train_loader:
  print(images.size())#バッチサイズ64のチャンネル３（RGB）32,32のデータ
  print(images[0].size())#画像一つのデータ
  print(labels.size())#一回のlabelsの数
  print(labels)
  break

torch.Size([64, 3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([64])
tensor([7, 6, 4, 7, 0, 0, 9, 9, 6, 3, 9, 3, 1, 4, 2, 8, 6, 0, 9, 9, 5, 7, 6, 9,
        0, 4, 9, 5, 9, 0, 6, 4, 7, 4, 1, 0, 4, 6, 2, 6, 8, 1, 8, 3, 6, 6, 8, 8,
        7, 6, 1, 2, 1, 9, 1, 6, 2, 8, 9, 0, 4, 7, 1, 2])


In [6]:
num_classes = 10

class MLPNet(nn.Module):
  def __init__(self):
    super(MLPNet , self).__init__()
    self.fc1 = nn.Linear(32 * 32 * 3 , 600)
    self.fc2 = nn.Linear(600 , 600)
    self.fc3 = nn.Linear(600 , num_classes)
    self.dropout1 = nn.Dropout2d(0.2)
    self.dropout2 = nn.Dropout2d(0.2)

  def forward(self ,x):
    x = F.relu(self.fc1(x))
    x = self.dropout1(x)
    x = F.relu(self.fc2(x))
    x = self.dropout2(x)
    return F.relu(self.fc3(x))

device = 'cuda' if torch.cuda.is_available() else 'cpu'

net = MLPNet().to(device)

In [7]:
device

'cuda'

In [9]:
#損失関数と最適化関数の設定
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters() , lr = 0.01 , momentum= 0.9 , weight_decay=5e-4)
